#Libraries and Data

In [1]:
#import libraries
import numpy as np
import pandas as pd
import itertools
from prophet import Prophet
import yfinance
import pickle
from ta import add_all_ta_features
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.stattools import adfuller
from matplotlib.pylab import rcParams
from sklearn import tree


/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
#load the data

data = yfinance.download (tickers = "TATAPOWER.NS",start="2017-03-06",
                               interval = "1d", group_by = 'ticker', auto_adjust = True)

data.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume
Date,,,,,
2023-07-07,229.800003,232.350006,226.100006,228.000000,16592198
2023-07-10,229.250000,229.250000,224.649994,225.000000,7143637
2023-07-11,226.000000,226.850006,224.699997,225.500000,5156417
2023-07-12,226.250000,227.250000,225.199997,226.300003,7366992
2023-07-13,227.000000,227.199997,220.899994,221.850006,9385696


#Data preparation

In [3]:
#transform the date variable
data["Date"] = pd.to_datetime(data.index, format = "%m/%d/%Y")


In [4]:
#renaming variables
data["y"] = data["Close"]
data = data.rename(columns = {
                     'Date': 'ds'})
data.head(10)
data = data.reset_index()

#Holidays

In [5]:
#Easter
#easter_dates = df[df.Easter == 1].ds
#easter = pd.DataFrame({'holiday': 'easter',
#                      'ds': pd.to_datetime(easter_dates),
#                      'lower_window': -5,
#                      'upper_window': 2})
#easter.head(2)

In [6]:
#Thanksgiving
#thanksgiving_dates = df[df.Thanksgiving == 1].ds
#thanksgiving = pd.DataFrame({'holiday': 'thanksgiving',
 #                      'ds': pd.to_datetime(thanksgiving_dates),
  #                     'lower_window': -5,
   #                    'upper_window': 2})
#thanksgiving.head(2)

In [7]:
#combining holidays
#holidays = pd.concat([easter, thanksgiving])
#holidays

In [8]:
data  = add_all_ta_features(data, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True)


/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


In [9]:
#create training and test set
test_days = 360
training_set = data.iloc[:-test_days,:]
test_set = data.iloc[-test_days:,:]

In [10]:
training_set

,Date,Open,High,Low,Close,Volume,ds,y,volume_adi,volume_obv,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
0,2017-03-06,68.984694,69.615652,68.900568,69.363266,4706563,2017-03-06,69.363266,1.384243e+06,4706563,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,69.363266,-37.730992,0.000000,0.000000
1,2017-03-07,69.741847,70.456931,69.573588,69.825974,7309780,2017-03-07,69.825974,-1.748489e+06,12016343,...,0.053188,0.010638,0.042550,4.238566,0.847713,3.390853,69.566793,0.667079,0.664863,0.667079
2,2017-03-08,70.036294,70.120421,68.942632,69.068825,3008752,2017-03-08,69.068825,-4.112503e+06,9007591,...,0.007222,0.009954,-0.002732,0.523082,0.782787,-0.259705,69.350968,-1.084337,-1.090259,-0.424492
3,2017-03-09,68.984701,69.531533,68.564063,69.152954,4968194,2017-03-09,69.152954,-3.032495e+06,13975785,...,-0.019219,0.004120,-0.023338,0.790092,0.784248,0.005844,69.264243,0.121805,0.121731,-0.303204
4,2017-03-10,69.237079,69.994223,68.395802,68.942627,1925931,2017-03-10,68.942627,-3.640692e+06,12049854,...,-0.063936,-0.009491,-0.054445,-4.280216,-0.228645,-4.051571,69.109924,-0.304148,-0.304611,-0.606429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207,2022-01-21,241.598281,246.512810,234.472213,237.814087,35507032,2022-01-21,237.814087,1.110350e+08,4564387284,...,2.650096,2.031511,0.618585,-2.511247,-9.474715,6.963468,238.362980,-2.694555,-2.731523,242.853070
1208,2022-01-24,238.354697,239.681626,222.726499,225.626068,43279408,2022-01-24,225.626068,8.255839e+07,4521107876,...,2.107775,2.046763,0.061011,0.281809,-7.523411,7.805220,238.263286,-5.125020,-5.261016,225.281783
1209,2022-01-25,222.382469,236.732901,219.188024,234.717941,39449718,2022-01-25,234.717941,1.129468e+08,4560557594,...,1.969074,2.031226,-0.062152,1.506523,-5.717424,7.223947,238.213228,4.029620,3.950548,238.389402
1210,2022-01-27,231.474343,238.207255,228.525625,236.929474,35920383,2022-01-27,236.929474,1.393857e+08,4596477977,...,1.913199,2.007620,-0.094421,1.627137,-4.248512,5.875649,238.201100,0.942209,0.937797,241.577736


#Prophet

In [11]:
m = Prophet(
            #holidays = holidays,
            #seasonality_mode = 'multiplicative',
            #seasonality_prior_scale = 20,
           # changepoint_prior_scale = 0.1,
           # holidays_prior_scale = 10
           )
m.add_regressor('volume_adi')
m.add_regressor('volume_obv')
m.add_regressor('volume_cmf')
m.add_regressor('others_dr')
m.add_regressor('volume_adi')
m.fit(training_set)


17:09:05 - cmdstanpy - INFO - Chain [1] start processing
17:09:06 - cmdstanpy - INFO - Chain [1] done processing


In [12]:
#create future df
#future = m.make_future_dataframe(periods = test_days,
 #                                freq = "D")

In [13]:
future = pd.DataFrame({"ds" : data.ds})

In [14]:
#forecast
forecast = m.predict(data)
forecast.tail()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,extra_regressors_additive,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
1567,2023-07-07,120.975496,190.721451,436.403895,-3.611552,242.602494,192.581818,192.581818,192.581818,204.354863,...,-0.228193,-0.228193,-0.228193,-11.544853,-11.544853,-11.544853,0.0,0.0,0.0,313.557314
1568,2023-07-10,121.370913,189.799978,433.908114,-3.663914,243.622157,192.036857,192.036857,192.036857,203.688369,...,-0.141983,-0.141983,-0.141983,-11.509529,-11.509529,-11.509529,0.0,0.0,0.0,313.407771
1569,2023-07-11,121.502719,187.831685,435.611819,-3.979284,244.378208,192.579551,192.579551,192.579551,204.208220,...,-0.154546,-0.154546,-0.154546,-11.474123,-11.474123,-11.474123,0.0,0.0,0.0,314.082270
1570,2023-07-12,121.634525,188.483667,437.975176,-4.294654,245.149009,193.149022,193.149022,193.149022,204.609460,...,-0.034979,-0.034979,-0.034979,-11.425458,-11.425458,-11.425458,0.0,0.0,0.0,314.783547
1571,2023-07-13,121.766330,183.645324,437.917419,-4.694396,246.028194,191.734965,191.734965,191.734965,203.222268,...,-0.124344,-0.124344,-0.124344,-11.362958,-11.362958,-11.362958,0.0,0.0,0.0,313.501296


In [15]:
data = data.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')
data = data.drop("index",axis=1)

In [16]:
#get some variables
prophet_variables = forecast.loc[:, ["trend", "weekly",  "multiplicative_terms","trend_upper","trend_lower"]]
df_xgb = pd.concat([data, prophet_variables], axis = 1)
df_xgb.head(1)

,Date,Open,High,Low,Close,Volume,ds,y,volume_adi,volume_obv,...,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr,trend,weekly,multiplicative_terms,trend_upper,trend_lower
0,2017-03-06,68.984694,69.615652,68.900568,69.363266,4706563,2017-03-06,69.363266,1.384243e+06,4706563,...,0.0,69.363266,-37.730992,0.0,0.0,111.517252,-0.141983,0.0,111.517252,111.517252


In [17]:
df_xgb  = add_all_ta_features(df_xgb, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True)


/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


In [18]:
    
non_stationaries = []
for col in df_xgb.columns:
    if col != "multiplicative_terms" and col != "Date" and col != "ds" :
        print(col)
        dftest = adfuller(df_xgb[col].values)
        p_value = dftest[1]
        t_test = dftest[0] < dftest[4]["1%"]
        if p_value > 0.05 or not t_test:
            non_stationaries.append(col)
print(f"Non-Stationary Features Found: {len(non_stationaries)}")

Open
High
Low
Close
Volume
y
volume_adi
volume_obv
volume_cmf
volume_fi
volume_em
volume_sma_em
volume_vpt
volume_vwap
volume_mfi
volume_nvi
volatility_bbm
volatility_bbh
volatility_bbl
volatility_bbw
volatility_bbp
volatility_bbhi
volatility_bbli
volatility_kcc
volatility_kch
volatility_kcl
volatility_kcw
volatility_kcp
volatility_kchi
volatility_kcli
volatility_dcl
volatility_dch
volatility_dcm
volatility_dcw
volatility_dcp
volatility_atr
volatility_ui
trend_macd
trend_macd_signal
trend_macd_diff
trend_sma_fast
trend_sma_slow
trend_ema_fast
trend_ema_slow
trend_vortex_ind_pos
trend_vortex_ind_neg
trend_vortex_ind_diff
trend_trix
trend_mass_index
trend_dpo
trend_kst
trend_kst_sig
trend_kst_diff
trend_ichimoku_conv
trend_ichimoku_base
trend_ichimoku_a
trend_ichimoku_b
trend_stc
trend_adx
trend_adx_pos
trend_adx_neg
trend_cci
trend_visual_ichimoku_a
trend_visual_ichimoku_b
trend_aroon_up
trend_aroon_down
trend_aroon_ind
trend_psar_up
trend_psar_down
trend_psar_up_indicator
trend_psar_do

In [19]:
df_stationary = df_xgb.copy()

df_stationary = df_stationary.drop(columns=["Date","ds"])
df_stationary

,Open,High,Low,Close,Volume,y,volume_adi,volume_obv,volume_cmf,volume_fi,...,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr,trend,weekly,multiplicative_terms,trend_upper,trend_lower
0,68.984694,69.615652,68.900568,69.363266,4706563,69.363266,1.384243e+06,4706563,0.294109,0.000000e+00,...,0.000000,69.363266,-37.730992,0.000000,0.000000,111.517252,-0.141983,0.0,111.517252,111.517252
1,69.741847,70.456931,69.573588,69.825974,7309780,69.825974,-1.748489e+06,12016343,-0.145509,3.382290e+06,...,3.390853,69.566793,0.667079,0.664863,0.667079,111.598375,-0.154546,0.0,111.598375,111.598375
2,70.036294,70.120421,68.942632,69.068825,3008752,69.068825,-4.112503e+06,9007591,-0.273709,2.573667e+06,...,-0.259705,69.350968,-1.084337,-1.090259,-0.424492,111.679498,-0.034979,0.0,111.679498,111.679498
3,68.984701,69.531533,68.564063,69.152954,4968194,69.152954,-3.032495e+06,13975785,-0.151676,2.265710e+06,...,0.005844,69.264243,0.121805,0.121731,-0.303204,111.760621,-0.124344,0.0,111.760621,111.760621
4,69.237079,69.994223,68.395802,68.942627,1925931,68.942627,-3.640692e+06,12049854,-0.166096,1.884169e+06,...,-4.051571,69.109924,-0.304148,-0.304611,-0.606429,111.841744,-0.228193,0.0,111.841744,111.841744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1567,229.800003,232.350006,226.100006,228.000000,16592198,228.000000,-2.037085e+08,5480198866,0.013058,2.791326e+07,...,15.857850,221.131214,-0.696867,-0.699306,228.704245,120.975496,-0.228193,0.0,242.602494,-3.611552
1568,229.250000,229.250000,224.649994,225.000000,7143637,225.000000,-2.097650e+08,5473055229,0.013168,2.086409e+07,...,9.356324,221.442765,-1.315789,-1.324523,224.379189,121.370913,-0.141983,0.0,243.622157,-3.663914
1569,226.000000,226.850006,224.699997,225.500000,5156417,225.500000,-2.110841e+08,5478211646,0.028744,1.825182e+07,...,3.420902,221.779041,0.222222,0.221976,225.100032,121.502719,-0.154546,0.0,244.378208,-3.979284
1570,226.250000,227.250000,225.199997,226.300003,7366992,226.300003,-2.105450e+08,5485578638,0.032625,1.648636e+07,...,0.831191,222.161683,0.354769,0.354141,226.253385,121.634525,-0.034979,0.0,245.149009,-4.294654


In [20]:
df_stationary[non_stationaries] = df_stationary[non_stationaries].pct_change()

#XGBoost preparation

In [21]:
#create training and test set
test_days = 360
training_set = df_xgb.iloc[:-test_days,:]
test_set = df_xgb.iloc[-test_days:,:]

In [22]:
#isolate X and y
y_train = training_set.y
y_test = test_set.y 

X_train = training_set.iloc[:, 2:]

X_test = test_set.iloc[:, 2:]


In [23]:
X_test

,High,Low,Close,Volume,ds,y,volume_adi,volume_obv,volume_cmf,volume_fi,...,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr,trend,weekly,multiplicative_terms,trend_upper,trend_lower
1212,245.284180,241.106830,241.844009,24939671,2022-01-31,241.844009,1.003128e+08,4655941503,0.066551,2.884054e+07,...,1.649239,238.235608,0.819508,0.816168,248.662950,52.172945,-0.141983,0.0,52.172945,52.172945
1213,252.017078,240.615378,245.579056,74185275,2022-02-01,245.579056,9.071994e+07,4730126778,0.023549,6.430410e+07,...,8.517939,238.289150,1.544403,1.532599,254.047711,52.304751,-0.154546,0.0,52.304751,52.304751
1214,250.641019,244.989309,246.316238,25162901,2022-02-02,246.316238,7.737272e+07,4755289679,-0.017392,5.776775e+07,...,3.902178,238.622704,0.300181,0.299732,255.110497,52.436556,-0.034979,0.0,52.436556,52.436556
1215,250.051263,245.038452,248.527771,27724870,2022-02-03,248.527771,8.824532e+07,4783014549,0.016184,5.827442e+07,...,1.161922,238.780145,0.897843,0.893836,258.298831,52.568362,-0.124344,0.0,52.568362,52.568362
1216,253.491440,248.675207,250.002136,39309420,2022-02-04,250.002136,7.059632e+07,4822323969,-0.037522,5.822899e+07,...,1.408124,239.008661,0.593240,0.591487,260.424401,52.700168,-0.228193,0.0,52.700168,52.700168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1567,232.350006,226.100006,228.000000,16592198,2023-07-07,228.000000,-2.037085e+08,5480198866,0.013058,2.791326e+07,...,15.857850,221.131214,-0.696867,-0.699306,228.704245,120.975496,-0.228193,0.0,242.602494,-3.611552
1568,229.250000,224.649994,225.000000,7143637,2023-07-10,225.000000,-2.097650e+08,5473055229,0.013168,2.086409e+07,...,9.356324,221.442765,-1.315789,-1.324523,224.379189,121.370913,-0.141983,0.0,243.622157,-3.663914
1569,226.850006,224.699997,225.500000,5156417,2023-07-11,225.500000,-2.110841e+08,5478211646,0.028744,1.825182e+07,...,3.420902,221.779041,0.222222,0.221976,225.100032,121.502719,-0.154546,0.0,244.378208,-3.979284
1570,227.250000,225.199997,226.300003,7366992,2023-07-12,226.300003,-2.105450e+08,5485578638,0.032625,1.648636e+07,...,0.831191,222.161683,0.354769,0.354141,226.253385,121.634525,-0.034979,0.0,245.149009,-4.294654


In [24]:
X_train = X_train.drop(columns=["ds"])
X_test = X_test.drop(columns=["ds"])
#X_test = StandardScaler().fit_transform(X_test)
#X_train = StandardScaler().fit_transform(X_train)
#y_train = StandardScaler().fit_transform(y_train)
#y_test = StandardScaler().fit_transform(y_test)


In [25]:
#Matrices
import xgboost as xgb
Train = xgb.DMatrix(X_train, label = y_train)
Test = xgb.DMatrix(X_test, label = y_test)

In [26]:
X_train

,High,Low,Close,Volume,y,volume_adi,volume_obv,volume_cmf,volume_fi,volume_em,...,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr,trend,weekly,multiplicative_terms,trend_upper,trend_lower
0,69.615652,68.900568,69.363266,4706563,69.363266,1.384243e+06,4706563,0.294109,0.000000e+00,0.000000,...,0.000000,69.363266,-37.730992,0.000000,0.000000,111.517252,-0.141983,0.0,111.517252,111.517252
1,70.456931,69.573588,69.825974,7309780,69.825974,-1.748489e+06,12016343,-0.145509,3.382290e+06,9.149704,...,3.390853,69.566793,0.667079,0.664863,0.667079,111.598375,-0.154546,0.0,111.598375,111.598375
2,70.120421,68.942632,69.068825,3008752,69.068825,-4.112503e+06,9007591,-0.273709,2.573667e+06,-18.935940,...,-0.259705,69.350968,-1.084337,-1.090259,-0.424492,111.679498,-0.034979,0.0,111.679498,111.679498
3,69.531533,68.564063,69.152954,4968194,69.152954,-3.032495e+06,13975785,-0.151676,2.265710e+06,-9.419777,...,0.005844,69.264243,0.121805,0.121731,-0.303204,111.760621,-0.124344,0.0,111.760621,111.760621
4,69.994223,68.395802,68.942627,1925931,68.942627,-3.640692e+06,12049854,-0.166096,1.884169e+06,12.218052,...,-4.051571,69.109924,-0.304148,-0.304611,-0.606429,111.841744,-0.228193,0.0,111.841744,111.841744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207,246.512810,234.472213,237.814087,35507032,237.814087,1.110350e+08,4564387284,0.127771,2.040738e+07,-132.490032,...,6.963468,238.362980,-2.694555,-2.731523,242.853070,50.854889,-0.228193,0.0,50.854889,50.854889
1208,239.681626,222.726499,225.626068,43279408,225.626068,8.255839e+07,4521107876,0.048944,-5.786371e+07,-363.884004,...,7.805220,238.263286,-5.125020,-5.261016,225.281783,51.250306,-0.141983,0.0,51.250306,51.250306
1209,236.732901,219.188024,234.717941,39449718,234.717941,1.129468e+08,4560557594,0.077093,1.641372e+06,-144.255923,...,7.223947,238.213228,4.029620,3.950548,238.389402,51.382111,-0.154546,0.0,51.382111,51.382111
1210,238.207255,228.525625,236.929474,35920383,236.929474,1.393857e+08,4596477977,0.135208,1.275533e+07,145.707425,...,5.875649,238.201100,0.942209,0.937797,241.577736,51.645723,-0.124344,0.0,51.645723,51.645723


In [42]:
#set the parameters
parameters = {
              'learning_rate': 0.1,
              'max_depth': 40,
              'colsample_bytree': 1,
              'subsample': .8,
              'min_child_weight' : 1,
              'gamma': 0,
              'random_state': 1600,
              'eval_metric': "rmse",
              'objective': "reg:squarederror"}

#XGBoost

In [44]:
model = xgb.train(params = parameters,
                  dtrain = Train,
                  num_boost_round = 200,
                  evals = [(Test, "y")])

[0]	y-rmse:197.50524
[1]	y-rmse:178.04162
[2]	y-rmse:160.44646
[3]	y-rmse:144.52112
[4]	y-rmse:129.98460
[5]	y-rmse:116.81157
[6]	y-rmse:105.26706
[7]	y-rmse:94.72305
[8]	y-rmse:85.29808
[9]	y-rmse:77.39339
[10]	y-rmse:69.58041
[11]	y-rmse:62.89316
[12]	y-rmse:56.97230
[13]	y-rmse:51.37380
[14]	y-rmse:46.66548
[15]	y-rmse:42.49702
[16]	y-rmse:38.57866
[17]	y-rmse:34.95823
[18]	y-rmse:31.91526
[19]	y-rmse:28.93963
[20]	y-rmse:26.29735
[21]	y-rmse:24.01984
[22]	y-rmse:22.00010
[23]	y-rmse:20.08082
[24]	y-rmse:18.35334
[25]	y-rmse:16.78890
[26]	y-rmse:15.38618
[27]	y-rmse:14.13056
[28]	y-rmse:13.07998
[29]	y-rmse:12.07782
[30]	y-rmse:11.18758
[31]	y-rmse:10.49725
[32]	y-rmse:9.83876
[33]	y-rmse:9.26076
[34]	y-rmse:8.69100
[35]	y-rmse:8.18468
[36]	y-rmse:7.73660
[37]	y-rmse:7.36405
[38]	y-rmse:7.01310
[39]	y-rmse:6.70657
[40]	y-rmse:6.50031
[41]	y-rmse:6.25699
[42]	y-rmse:6.04449
[43]	y-rmse:5.86331
[44]	y-rmse:5.75188
[45]	y-rmse:5.60132
[46]	y-rmse:5.47641
[47]	y-rmse:5.40164
[48]	y-rmse

In [ ]:
dir(model)
print(model.get_score()) 

In [ ]:
#forecasting
predictions = pd.Series(model.predict(Test), name = "XGBoost")
predictions.index  =test_set.ds

#VIZ and assessment

In [ ]:
#Visualization
training_set.index = training_set.ds
test_set.index = test_set.ds

In [ ]:
training_set.y['2022-01-01':].plot(figsize = (9,6), legend = True)
test_set.y.plot(legend = True)
predictions.plot(legend = True)

In [ ]:
#MAPE
def MAPE(y_true, y_pred):
  y_true, y_pred = np.array(y_true), np.array(y_pred)
  return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
MAPE(test_set.y, predictions)

In [ ]:
#rcParams["figure.figsize"] = 80, 50
#tree.plot_tree(model)

#Future

